In [ ]:
# unzip dataset
!unzip "dataset_bayam.zip"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import tensorflow as tf
import numpy as np
import random

base_dir = './dataset_bayam'

# Set seed for reproducibility
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Membuat ImageDataGenerator untuk pelatihan dan validasi
# Digunakan untuk mempersiapkan data augmentasi yang membuat model lebih kuat dan mampu mengenali berbagai variasi gambar.
# Data ini kemudian akan digunakan dalam proses pelatihan dan validasi model machine learning.
train_datagen = ImageDataGenerator( # Membuat sebuah instance dari kelas ImageDataGenerator yang akan digunakan untuk menghasilkan data gambar dengan augmentasi.
    rescale=1./255, # Menormalisasi nilai piksel gambar dari rentang 0-255 menjadi rentang 0-1. Ini dilakukan dengan membagi setiap nilai piksel dengan 255.
    rotation_range=20,  # Mengacak rotasi gambar hingga maksimal 20 derajat ke kanan atau kiri. Hal ini membantu model menjadi lebih tahan terhadap variasi rotasi.
    width_shift_range=0.2,  # Menggeser gambar secara horizontal (ke kiri atau kanan) hingga maksimal 20% dari lebar gambar asli. Ini membuat model lebih robust terhadap variasi posisi horizontal.
    height_shift_range=0.2, # Menggeser gambar secara vertikal (ke atas atau bawah) hingga maksimal 20% dari tinggi gambar asli. Ini membantu model beradaptasi dengan variasi posisi vertikal.
    horizontal_flip=True, # Membalik gambar secara horizontal secara acak. Ini berguna untuk menambahkan variasi pada data pelatihan.
    shear_range=0.2,  # Menerapkan transformasi shear (kemiringan) pada gambar hingga maksimal 20%. Ini membuat model lebih tahan terhadap distorsi bentuk.
    zoom_range=0.2, # Memperbesar atau memperkecil gambar hingga maksimal 20%. Ini membantu model beradaptasi dengan variasi skala.
    fill_mode='nearest',  # Menentukan bagaimana piksel yang hilang setelah transformasi (misalnya setelah rotasi atau geseran) akan diisi. Dalam hal ini, piksel akan diisi dengan nilai piksel terdekat.
    validation_split=0.2  # membagi 80% untuk pelatiahn dan 20% dari data untuk validasi
)

# Membuat ImageDataGenerator untuk testing
test_datagen = ImageDataGenerator(rescale=1./255)

# membagi data menjadi train dataset dan validation dataset
# Generator data pelatihan
train_generator = train_datagen.flow_from_directory(  # Fungsi ini membuat generator data dari gambar yang disimpan dalam folder.
    base_dir,  # Direktori sumber data
    target_size=(320, 320),  # Mengubah resolusi seluruh gambar menjadi 320x320 piksel
    batch_size=32,  # Gambar-gambar diambil dalam kelompok (batch) berukuran 32 gambar setiap kali.
    class_mode='categorical',  # Karena ini adalah masalah klasifikasi multi-klas
    subset='training',  # Subset pelatihan
    seed=seed  # Set seed for train generator
)

# Generator data validasi
validation_generator = train_datagen.flow_from_directory(
    base_dir,  # Direktori sumber data
    target_size=(320, 320),  # Mengubah resolusi seluruh gambar menjadi 320x320 piksel
    batch_size=32,
    class_mode='categorical',  # Karena ini adalah masalah klasifikasi multi-klas
    subset='validation',  # Subset validasi
    seed=seed  # Set seed for train generator
)

print("Ukuran batch training: ", train_generator.samples)
print("Ukuran batch validation: ", validation_generator.samples)

# Load beberapa gambar asli
original_images = []
for i in range(10):  # Misalnya 10 gambar
    img = load_img(train_generator.filepaths[i], target_size=(320, 320)) # load gambar sesuai alamat dan ubah menjadi ukuran 320x320
    img = img_to_array(img) / 255.0  # Normalisasi gambar # konversi objek gambar menjadi array numpy dan lakukan normalisasi agar dapat ditampilkan
    original_images.append(img)

# Ambil satu batch gambar yang sudah dilakukan augmentasi
images, labels = next(train_generator)

# Fungsi untuk menampilkan gambar
import matplotlib.pyplot as plt

def plot_images(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# Tampilkan gambar-gambar yang telah di-augmentasi
print("Sebelum augmentasi: ")
plot_images(original_images)

# Tampilkan gambar-gambar yang telah di-augmentasi
print("Hasil augmentasi: ")
plot_images(images[:10])

# 1. Self-made model

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Input layer
    tf.keras.layers.Conv2D(32, (3, 3), input_shape=(320, 320, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # Second conv layer
    tf.keras.layers.Conv2D(64, (3, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # Third conv layer
    tf.keras.layers.Conv2D(128, (3, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # Classification layers
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary to verify the configuration
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, Callback

# Menginisialisasi callback EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.1,
    patience=10,
    verbose=1,
    mode='max',
    restore_best_weights=True
)

# Latih model dengan model.fit
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  # berapa batch yang akan dieksekusi pada setiap epoch
    epochs=100,  # Tambahkan epochs jika akurasi model belum optimal
    validation_data=validation_generator,  # Menampilkan akurasi pengujian data validasi
    validation_steps=validation_generator.samples // validation_generator.batch_size,  # berapa batch yang akan dieksekusi pada setiap epoch
    # callbacks=[early_stopping],
    verbose=2,
)

In [ ]:
# Visualisasi Model
from tensorflow.keras.utils import plot_model

# Menyimpan arsitektur model ke dalam file PNG
plot_model(model, to_file='model_architecture_self-made.png', show_shapes=True, show_layer_names=True)

# Menampilkan arsitektur model di notebook Colab
from IPython.display import Image
Image('model_architecture_self-made.png')

In [ ]:
import matplotlib.pyplot as plt

# Plot akurasi pelatihan dan validasi
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Menampilkan akurasi terakhir dari data validasi
print("Akurasi terakhir:", history.history['accuracy'][-1])
print("Akurasi validasi terakhir:", history.history['val_accuracy'][-1])

In [ ]:
import matplotlib.pyplot as plt

# Plot loss pelatihan dan validasi
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Menampilkan loss terakhir dari data validasi
print("Loss terakhir:", history.history['loss'][-1])
print("Loss validasi terakhir:", history.history['val_loss'][-1])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
from math import ceil

# Membuat generator validasi baru tanpa shuffle untuk evaluasi
validation_generator_for_evaluation = train_datagen.flow_from_directory(
    base_dir,
    target_size=(320, 320),
    batch_size=32,  # Sesuai dengan batch size sebelumnya
    class_mode='categorical',
    subset='validation',
    shuffle=False  # Penting: non-aktifkan shuffle
)

# Perbaikan jumlah steps untuk memastikan semua sampel diproses
steps = ceil(validation_generator_for_evaluation.samples / validation_generator_for_evaluation.batch_size)

# Memprediksi seluruh data validasi dengan perbaikan steps
predictions = model.predict(validation_generator_for_evaluation, steps=steps)
predicted_labels = np.argmax(predictions, axis=1)

# Mendapatkan label sebenarnya dari generator
true_labels = validation_generator_for_evaluation.classes

# Memastikan jumlah prediksi sama dengan label sebenarnya
assert predicted_labels.shape[0] == true_labels.shape[0], "Jumlah prediksi dan label sebenarnya harus sama."

# Menghitung confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Menghitung metrik lain
report = classification_report(true_labels, predicted_labels, target_names=validation_generator_for_evaluation.class_indices.keys())

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(report)

# Menampilkan class indices
class_indices = validation_generator_for_evaluation.class_indices
print("Class indices:", class_indices)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Menampilkan confusion matrix sebagai heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Karat Putih', 'Kekurangan Mangan', 'Sehat', 'Virus Keriting' ], yticklabels=['Karat Putih', 'Kekurangan Mangan', 'Sehat', 'Virus Keriting'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import tensorflow as tf

# Simpan model dalam format TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Mengaktifkan quantization
tflite_model = converter.convert()

# Simpan model TFLite ke file
tflite_model_file = 'model_self-made.tflite'
with open(tflite_model_file, 'wb') as f:
    f.write(tflite_model)

# Mengetahui ukuran model TFLite
import os

# Fungsi untuk mengonversi bytes ke megabytes
def bytes_to_mb(size_in_bytes):
    size_in_mb = size_in_bytes / (1024 * 1024)
    return size_in_mb

# Mengetahui ukuran model TFLite dalam bytes dan MB
file_size_in_bytes = os.path.getsize(tflite_model_file)
file_size_in_mb = bytes_to_mb(file_size_in_bytes)
print(f"Ukuran model TFLite setelah quantization: {file_size_in_bytes} bytes ({file_size_in_mb:.2f} MB)")

# 2. Transfer Learning using MobileNetV2

In [ ]:
# Menggunakan MobileNetV2 untuk transfer learning
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load pre-trained MobileNetV2 tanpa lapisan atas
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(320, 320, 3))

# Tambahkan lapisan baru di atasnya
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)  # Misal kita punya 4 kelas

# Buat model baru
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze semua lapisan dari base_model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary to verify the configuration
model.summary()

In [ ]:
# Visualisasi Model
from tensorflow.keras.utils import plot_model

# Menyimpan arsitektur model ke dalam file PNG
plot_model(model, to_file='model_architecture_transfer_learning_mobilenetv2.png', show_shapes=True, show_layer_names=True)

# Menampilkan arsitektur model di notebook Colab
from IPython.display import Image
Image('model_architecture_transfer_learning_mobilenetv2.png')

In [ ]:
# Latih model dengan model.fit
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  # berapa batch yang akan dieksekusi pada setiap epoch
    epochs=100,  # Tambahkan epochs jika akurasi model belum optimal
    validation_data=validation_generator,  # Menampilkan akurasi pengujian data validasi
    validation_steps=validation_generator.samples // validation_generator.batch_size,  # berapa batch yang akan dieksekusi pada setiap epoch
    # callbacks=[early_stopping],
    verbose=2,
)

In [ ]:
# Mengevaluasi model
loss, accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print("Akurasi pada data validasi:", accuracy)

In [ ]:
print(history.history.keys())

In [ ]:
import matplotlib.pyplot as plt

# Plot akurasi pelatihan dan validasi
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Menampilkan akurasi terakhir dari data validasi
print("Akurasi terakhir:", history.history['accuracy'][-1])
print("Akurasi validasi terakhir:", history.history['val_accuracy'][-1])

In [ ]:
import matplotlib.pyplot as plt

# Plot loss pelatihan dan validasi
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Menampilkan loss terakhir dari data validasi
print("Loss terakhir:", history.history['loss'][-1])
print("Loss validasi terakhir:", history.history['val_loss'][-1])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
from math import ceil

# Perbaikan jumlah steps untuk memastikan semua sampel diproses
steps = ceil(validation_generator_for_evaluation.samples / validation_generator_for_evaluation.batch_size)

# Memprediksi seluruh data validasi dengan perbaikan steps
predictions = model.predict(validation_generator_for_evaluation, steps=steps)
predicted_labels = np.argmax(predictions, axis=1)

# Mendapatkan label sebenarnya dari generator
true_labels = validation_generator_for_evaluation.classes

# Memastikan jumlah prediksi sama dengan label sebenarnya
assert predicted_labels.shape[0] == true_labels.shape[0], "Jumlah prediksi dan label sebenarnya harus sama."

# Menghitung confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Menghitung metrik lain
report = classification_report(true_labels, predicted_labels, target_names=validation_generator_for_evaluation.class_indices.keys())

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(report)

# Menampilkan class indices
class_indices = validation_generator_for_evaluation.class_indices
print("Class indices:", class_indices)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Menampilkan confusion matrix sebagai heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Karat Putih', 'Kekurangan Mangan', 'Sehat', 'Virus Keriting' ], yticklabels=['Karat Putih', 'Kekurangan Mangan', 'Sehat', 'Virus Keriting'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import tensorflow as tf

# Simpan model dalam format TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Mengaktifkan quantization
tflite_model = converter.convert()

# Simpan model TFLite ke file
tflite_model_file = 'model_MobileNetV2.tflite'
with open(tflite_model_file, 'wb') as f:
    f.write(tflite_model)

# Mengetahui ukuran model TFLite
import os

# Fungsi untuk mengonversi bytes ke megabytes
def bytes_to_mb(size_in_bytes):
    size_in_mb = size_in_bytes / (1024 * 1024)
    return size_in_mb

# Mengetahui ukuran model TFLite dalam bytes dan MB
file_size_in_bytes = os.path.getsize(tflite_model_file)
file_size_in_mb = bytes_to_mb(file_size_in_bytes)
print(f"Ukuran model TFLite setelah quantization: {file_size_in_bytes} bytes ({file_size_in_mb:.2f} MB)")

# 3. Transfer Learning using InceptionV3

In [ ]:
# Menggunakan InceptionV3 untuk transfer learning
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load pre-trained InceptionV3 tanpa lapisan atas
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(320, 320, 3))

# Tambahkan lapisan baru di atasnya
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)  # Misal kita punya 4 kelas

# Buat model baru
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze semua lapisan dari base_model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary to verify the configuration
model.summary()

In [ ]:
# Visualisasi Model
from tensorflow.keras.utils import plot_model

# Menyimpan arsitektur model ke dalam file PNG
plot_model(model, to_file='model_architecture_inceptionV3.png', show_shapes=True, show_layer_names=True)

# Menampilkan arsitektur model di notebook Colab
from IPython.display import Image
Image('model_architecture_inceptionV3.png')

In [ ]:
# Latih model dengan model.fit
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  # berapa batch yang akan dieksekusi pada setiap epoch
    epochs=100,  # Tambahkan epochs jika akurasi model belum optimal
    validation_data=validation_generator,  # Menampilkan akurasi pengujian data validasi
    validation_steps=validation_generator.samples // validation_generator.batch_size,  # berapa batch yang akan dieksekusi pada setiap epoch
    # callbacks=[early_stopping],
    verbose=2,
)

In [ ]:
# Mengevaluasi model
loss, accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print("Akurasi pada data validasi:", accuracy)

In [ ]:
print(history.history.keys())

In [ ]:
import matplotlib.pyplot as plt

# Plot akurasi pelatihan dan validasi
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Menampilkan akurasi terakhir dari data validasi
print("Akurasi terakhir:", history.history['accuracy'][-1])
print("Akurasi validasi terakhir:", history.history['val_accuracy'][-1])

In [ ]:
import matplotlib.pyplot as plt

# Plot loss pelatihan dan validasi
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Menampilkan loss terakhir dari data validasi
print("Loss terakhir:", history.history['loss'][-1])
print("Loss validasi terakhir:", history.history['val_loss'][-1])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
from math import ceil

# Perbaikan jumlah steps untuk memastikan semua sampel diproses
steps = ceil(validation_generator_for_evaluation.samples / validation_generator_for_evaluation.batch_size)

# Memprediksi seluruh data validasi dengan perbaikan steps
predictions = model.predict(validation_generator_for_evaluation, steps=steps)
predicted_labels = np.argmax(predictions, axis=1)

# Mendapatkan label sebenarnya dari generator
true_labels = validation_generator_for_evaluation.classes

# Memastikan jumlah prediksi sama dengan label sebenarnya
assert predicted_labels.shape[0] == true_labels.shape[0], "Jumlah prediksi dan label sebenarnya harus sama."

# Menghitung confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Menghitung metrik lain
report = classification_report(true_labels, predicted_labels, target_names=validation_generator_for_evaluation.class_indices.keys())

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(report)

# Menampilkan class indices
class_indices = validation_generator_for_evaluation.class_indices
print("Class indices:", class_indices)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Menampilkan confusion matrix sebagai heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Karat Putih', 'Kekurangan Mangan', 'Sehat', 'Virus Keriting' ], yticklabels=['Karat Putih', 'Kekurangan Mangan', 'Sehat', 'Virus Keriting'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import tensorflow as tf

# Simpan model dalam format TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Mengaktifkan quantization
tflite_model = converter.convert()

# Simpan model TFLite ke file
tflite_model_file = 'model_InceptionV3.tflite'
with open(tflite_model_file, 'wb') as f:
    f.write(tflite_model)

# Mengetahui ukuran model TFLite
import os

# Fungsi untuk mengonversi bytes ke megabytes
def bytes_to_mb(size_in_bytes):
    size_in_mb = size_in_bytes / (1024 * 1024)
    return size_in_mb

# Mengetahui ukuran model TFLite dalam bytes dan MB
file_size_in_bytes = os.path.getsize(tflite_model_file)
file_size_in_mb = bytes_to_mb(file_size_in_bytes)
print(f"Ukuran model TFLite setelah quantization: {file_size_in_bytes} bytes ({file_size_in_mb:.2f} MB)")

# 4. Transfer Learning using ResNet50

In [ ]:
# Menggunakan ResNet50 untuk transfer learning
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load pre-trained ResNet50 tanpa lapisan atas
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(320, 320, 3))

# Tambahkan lapisan baru di atasnya
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)  # Misal kita punya 4 kelas

# Buat model baru
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze semua lapisan dari base_model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary to verify the configuration
model.summary()

In [ ]:
# Visualisasi Model
from tensorflow.keras.utils import plot_model

# Menyimpan arsitektur model ke dalam file PNG
plot_model(model, to_file='model_architecture_resnet50.png', show_shapes=True, show_layer_names=True)

# Menampilkan arsitektur model di notebook Colab
from IPython.display import Image
Image('model_architecture_resnet50.png')

# Latih model dengan model.fit
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  # berapa batch yang akan dieksekusi pada setiap epoch
    epochs=100,  # Tambahkan epochs jika akurasi model belum optimal
    validation_data=validation_generator,  # Menampilkan akurasi pengujian data validasi
    validation_steps=validation_generator.samples // validation_generator.batch_size,  # berapa batch yang akan dieksekusi pada setiap epoch
    verbose=2,
)

In [ ]:
# Mengevaluasi model
loss, accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print("Akurasi pada data validasi:", accuracy)

In [ ]:
print(history.history.keys())

In [ ]:
import matplotlib.pyplot as plt

# Plot akurasi pelatihan dan validasi
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Menampilkan akurasi terakhir dari data validasi
print("Akurasi terakhir:", history.history['accuracy'][-1])
print("Akurasi validasi terakhir:", history.history['val_accuracy'][-1])

In [ ]:
import matplotlib.pyplot as plt

# Plot loss pelatihan dan validasi
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Menampilkan loss terakhir dari data validasi
print("Loss terakhir:", history.history['loss'][-1])
print("Loss validasi terakhir:", history.history['val_loss'][-1])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
from math import ceil

# Perbaikan jumlah steps untuk memastikan semua sampel diproses
steps = ceil(validation_generator_for_evaluation.samples / validation_generator_for_evaluation.batch_size)

# Memprediksi seluruh data validasi dengan perbaikan steps
predictions = model.predict(validation_generator_for_evaluation, steps=steps)
predicted_labels = np.argmax(predictions, axis=1)

# Mendapatkan label sebenarnya dari generator
true_labels = validation_generator_for_evaluation.classes

# Memastikan jumlah prediksi sama dengan label sebenarnya
assert predicted_labels.shape[0] == true_labels.shape[0], "Jumlah prediksi dan label sebenarnya harus sama."

# Menghitung confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Menghitung metrik lain
report = classification_report(true_labels, predicted_labels, target_names=validation_generator_for_evaluation.class_indices.keys())

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(report)

# Menampilkan class indices
class_indices = validation_generator_for_evaluation.class_indices
print("Class indices:", class_indices)

In [ ]:
# Mengubah model ke format TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Mengaktifkan quantization
tflite_model = converter.convert()

# Simpan model TFLite ke file
tflite_model_file = 'model_resnet50_quantized.tflite'
with open(tflite_model_file, 'wb') as f:
    f.write(tflite_model)

# Fungsi untuk mengonversi bytes ke megabytes
def bytes_to_mb(size_in_bytes):
    size_in_mb = size_in_bytes / (1024 * 1024)
    return size_in_mb

# Mengetahui ukuran model TFLite dalam bytes dan MB
file_size_in_bytes = os.path.getsize(tflite_model_file)
file_size_in_mb = bytes_to_mb(file_size_in_bytes)
print(f"Ukuran model TFLite setelah quantization: {file_size_in_bytes} bytes ({file_size_in_mb:.2f} MB)")

# 5. Transfer Learning using DenseNet121

In [ ]:
# Menggunakan DenseNet121 untuk transfer learning
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load pre-trained DenseNet121 tanpa lapisan atas
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(320, 320, 3))

# Tambahkan lapisan baru di atasnya
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)  # Misal kita punya 4 kelas

# Buat model baru
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze semua lapisan dari base_model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary to verify the configuration
model.summary()

In [ ]:
# Visualisasi Model
from tensorflow.keras.utils import plot_model

# Menyimpan arsitektur model ke dalam file PNG
plot_model(model, to_file='model_architecture_densenet121.png', show_shapes=True, show_layer_names=True)

# Menampilkan arsitektur model di notebook Colab
from IPython.display import Image
Image('model_architecture_densenet121.png')

In [ ]:
# Latih model dengan model.fit
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  # berapa batch yang akan dieksekusi pada setiap epoch
    epochs=100,  # Tambahkan epochs jika akurasi model belum optimal
    validation_data=validation_generator,  # Menampilkan akurasi pengujian data validasi
    validation_steps=validation_generator.samples // validation_generator.batch_size,  # berapa batch yang akan dieksekusi pada setiap epoch
    verbose=2,
)

In [ ]:
# Mengevaluasi model
loss, accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print("Akurasi pada data validasi:", accuracy)

In [ ]:
print(history.history.keys())

In [ ]:
import matplotlib.pyplot as plt

# Plot akurasi pelatihan dan validasi
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Menampilkan akurasi terakhir dari data validasi
print("Akurasi terakhir:", history.history['accuracy'][-1])
print("Akurasi validasi terakhir:", history.history['val_accuracy'][-1])

In [ ]:
import matplotlib.pyplot as plt

# Plot loss pelatihan dan validasi
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Menampilkan loss terakhir dari data validasi
print("Loss terakhir:", history.history['loss'][-1])
print("Loss validasi terakhir:", history.history['val_loss'][-1])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
from math import ceil

# Perbaikan jumlah steps untuk memastikan semua sampel diproses
steps = ceil(validation_generator_for_evaluation.samples / validation_generator_for_evaluation.batch_size)

# Memprediksi seluruh data validasi dengan perbaikan steps
predictions = model.predict(validation_generator_for_evaluation, steps=steps)
predicted_labels = np.argmax(predictions, axis=1)

# Mendapatkan label sebenarnya dari generator
true_labels = validation_generator_for_evaluation.classes

# Memastikan jumlah prediksi sama dengan label sebenarnya
assert predicted_labels.shape[0] == true_labels.shape[0], "Jumlah prediksi dan label sebenarnya harus sama."

# Menghitung confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Menghitung metrik lain
report = classification_report(true_labels, predicted_labels, target_names=validation_generator_for_evaluation.class_indices.keys())

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(report)

# Menampilkan class indices
class_indices = validation_generator_for_evaluation.class_indices
print("Class indices:", class_indices)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Menampilkan confusion matrix sebagai heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Karat Putih', 'Kekurangan Mangan', 'Sehat', 'Virus Keriting' ], yticklabels=['Karat Putih', 'Kekurangan Mangan', 'Sehat', 'Virus Keriting'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Mengubah model ke format TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Mengaktifkan quantization
tflite_model = converter.convert()

# Simpan model TFLite ke file
tflite_model_file = 'model_densenet121_quantized.tflite'
with open(tflite_model_file, 'wb') as f:
    f.write(tflite_model)

# Fungsi untuk mengonversi bytes ke megabytes
def bytes_to_mb(size_in_bytes):
    size_in_mb = size_in_bytes / (1024 * 1024)
    return size_in_mb

# Mengetahui ukuran model TFLite dalam bytes dan MB
file_size_in_bytes = os.path.getsize(tflite_model_file)
file_size_in_mb = bytes_to_mb(file_size_in_bytes)
print(f"Ukuran model TFLite setelah quantization: {file_size_in_bytes} bytes ({file_size_in_mb:.2f} MB)")